<a href="https://colab.research.google.com/github/ror74559/house_price_kaggle/blob/main/House_Price(kaggle).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Prices - Advanced Regression Techniques(kaggle)

Preveja preços de venda e pratique engenharia de recursos, RFs e aumento de gradiente



#Importando bibliotecas

In [60]:
import pandas as pd


#Leitura do Dataset

In [61]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQv4gEKgTDJ1ASHI0fq3cSEbpX2wlHi461M-Ts5pDOaHI0BLKpQBZmhzxNEbPLX_WsXvCNAFOgumcvs/pub?output=csv')
df_t =  pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRmxhGa2NKjE-y0rIBmdisP-LUuFzChmuDo8ueaJGwxadx1Vy4K-_lAgCBH1FrdjLtK6NPTxfY5GuR-/pub?output=csv')


#Análise exploratória dos dados

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

O dataset possui muitas colunas e também muitas colunas nulas, com isso vou criar um dicionário com apenas as colunas que possuem registros nulos e respectiva quantidade de registros

##Criando um dicionário com o nome das colunas com registros nulos 

In [63]:
def dict_nulos(dataset):
  nulos = dataset.isna().sum()
  dict_nulos = {}
  for i in range(len(nulos)):
    if nulos[i] != 0:
      dict_nulos[nulos.index[i]] = nulos[i]
  return dict_nulos


In [64]:
dicionario_nulos = dict_nulos(df)
dicionario_nulos

{'Alley': 1369,
 'BsmtCond': 37,
 'BsmtExposure': 38,
 'BsmtFinType1': 37,
 'BsmtFinType2': 38,
 'BsmtQual': 37,
 'Electrical': 1,
 'Fence': 1179,
 'FireplaceQu': 690,
 'GarageCond': 81,
 'GarageFinish': 81,
 'GarageQual': 81,
 'GarageType': 81,
 'GarageYrBlt': 81,
 'LotFrontage': 259,
 'MasVnrArea': 8,
 'MasVnrType': 8,
 'MiscFeature': 1406,
 'PoolQC': 1453}

#Criando função para dropar as colunas com mais de 1000 registros nulos.

In [65]:
def drop_cols(dic_nome_nulos,dataset):
  for item in dic_nome_nulos:
    if dic_nome_nulos[item] >1000:
      dataset.drop(columns=item, axis=1, inplace=True)
  print('ok')



In [66]:
drop_cols(dicionario_nulos,df)

ok


In [67]:
dicionario_nulos = dict_nulos(df)
dicionario_nulos

{'BsmtCond': 37,
 'BsmtExposure': 38,
 'BsmtFinType1': 37,
 'BsmtFinType2': 38,
 'BsmtQual': 37,
 'Electrical': 1,
 'FireplaceQu': 690,
 'GarageCond': 81,
 'GarageFinish': 81,
 'GarageQual': 81,
 'GarageType': 81,
 'GarageYrBlt': 81,
 'LotFrontage': 259,
 'MasVnrArea': 8,
 'MasVnrType': 8}

colunas deletadas

#Criar função para separar colunas categóricas das não categóricas

In [68]:
def sep_cat(dic_nome_nulos,dataset):
  cat = []
  nao_cat = []
  for item in dic_nome_nulos:
    if dataset[item].dtype == 'int64' or dataset[item].dtype == 'float64':
      nao_cat.append(item)
    else:
      cat.append(item)
  print('ok')
  return cat, nao_cat




A função retorna duas listas categorica, nao categorica.

In [69]:
categorico, nao_categorico = sep_cat(dicionario_nulos,df)


ok


In [70]:
categorico

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [71]:
nao_categorico

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

#Criar função para preencher nulos com a moda no caso de categorica ou booleano e para preencher nulos com a mediana no caso de não categórica.

In [72]:
def preencher_nulos_cat(dataset, col_categorica):
  for coluna in col_categorica:
    dataset[coluna] = dataset[coluna].fillna(dataset[coluna].mode(0)[0])   

In [73]:
preencher_nulos_cat(df,categorico)

In [74]:
def preencher_nulos_nao_cat(dataset,categorica):
  for coluna in categorica:
    dataset[coluna] = dataset[coluna].fillna(dataset[coluna].median())  

In [75]:
preencher_nulos_nao_cat(df,nao_categorico)

In [76]:
dicionario_nulos = dict_nulos(df)
dicionario_nulos

{'FireplaceQu': 690}

In [77]:
df['FireplaceQu'].mode()

0    Gd
dtype: object

In [78]:
df['FireplaceQu'] = df['FireplaceQu'].fillna('Gd')

In [79]:
dicionario_nulos = dict_nulos(df)
dicionario_nulos

{}

colunas preenchidas

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

#Classificar as categorias numericamente

In [81]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Gd,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [82]:
def sep_cat_total(dataset):
  cat = []
  for item in dataset.columns:
    if dataset[item].dtype != 'int64' and dataset[item].dtype != 'float64':
      cat.append(item)
  print('ok')
  return cat

In [83]:
categorico_total = sep_cat_total(df)

ok


In [84]:
categorico_total

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [85]:
porcentagem_categorico = (len(categorico_total)/len(df.columns))*100
porcentagem_categorico

50.649350649350644

In [86]:
def classif_categoria(dataset, categoria):
  for item in categoria:
    troca ={}
    cat = dataset[item].value_counts()
    for i in range(len(cat)):
      troca[cat.index[i]] = i
      dataset[item] = dataset[item].replace(troca)
  print('ok')




In [87]:
classif_categoria(df,categorico_total)

ok


In [88]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,0,1,0,0,0,1,7,5,2003,2003,0,0,0,0,1,196.0,1,0,0,1,0,0,1,706,0,0,150,856,0,0,0,0,856,854,0,1710,1,0,2,1,3,1,1,8,0,0,0,0,2003.0,1,2,548,0,0,0,0,61,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,3,0,22,1,0,0,0,6,8,1976,1976,0,0,2,1,0,0.0,0,0,1,1,0,2,2,978,0,0,284,1262,0,0,0,0,1262,0,0,1262,0,1,2,0,3,1,0,6,0,1,1,0,1976.0,1,2,460,0,0,0,298,0,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,1,0,0,0,0,1,0,0,0,1,7,5,2001,2002,0,0,0,0,1,162.0,1,0,0,1,0,3,1,486,0,0,434,920,0,0,0,0,920,866,0,1786,1,0,2,1,3,1,1,6,0,1,1,0,2001.0,1,2,608,0,0,0,0,42,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,1,0,0,1,0,11,0,0,0,1,7,5,1915,1970,0,0,3,6,0,0.0,0,0,2,0,1,0,2,216,0,0,540,756,0,2,0,0,961,756,0,1717,1,0,1,0,3,1,1,7,0,1,0,1,1998.0,0,3,642,0,0,0,0,35,272,0,0,0,0,2,2006,0,2,140000
4,5,60,0,84.0,14260,0,1,0,0,3,0,13,0,0,0,1,8,5,2000,2000,0,0,0,0,1,350.0,1,0,0,1,0,1,1,655,0,0,490,1145,0,0,0,0,1145,1053,0,2198,1,0,2,1,4,1,1,9,0,1,1,0,2000.0,1,3,836,0,0,0,192,84,0,0,0,0,0,12,2008,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,0,5,0,0,0,1,6,5,1999,2000,0,0,0,0,0,0.0,0,0,0,1,0,0,0,0,0,0,953,953,0,0,0,0,953,694,0,1647,0,0,2,1,3,1,0,7,0,1,1,0,1999.0,1,2,460,0,0,0,0,40,0,0,0,0,0,8,2007,0,0,175000
1456,1457,20,0,85.0,13175,0,0,0,0,0,0,8,0,0,0,0,6,6,1978,1988,0,0,4,4,2,119.0,0,0,1,1,0,0,2,790,1,163,589,1542,0,1,0,0,2073,0,0,2073,1,0,2,0,3,1,0,7,2,2,1,0,1978.0,0,2,500,0,0,0,349,0,0,0,0,0,0,2,2010,0,0,210000
1457,1458,70,0,66.0,9042,0,0,0,0,0,0,11,0,0,0,1,7,9,1941,2006,0,0,5,5,0,0.0,2,1,4,0,1,0,1,275,0,0,877,1152,0,0,0,0,1188,1152,0,2340,0,0,2,0,4,1,1,9,0,2,0,0,1941.0,1,1,252,0,0,0,0,60,0,0,0,0,2500,5,2010,0,0,266500
1458,1459,20,0,68.0,9717,0,0,0,0,0,0,0,0,0,0,0,5,6,1950,1996,1,0,2,1,0,0.0,0,0,1,0,0,3,1,49,1,1029,0,1078,0,2,0,1,1078,0,0,1078,1,0,1,0,2,1,1,5,0,0,0,0,1950.0,0,1,240,0,0,0,366,0,112,0,0,0,0,4,2010,0,0,142125


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   int64  
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   int64  
 10  LandSlope      1460 non-null   int64  
 11  Neighborhood   1460 non-null   int64  
 12  Condition1     1460 non-null   int64  
 13  Condition2     1460 non-null   int64  
 14  BldgType       1460 non-null   int64  
 15  HouseStyle     1460 non-null   int64  
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

Dataset pronto

In [90]:
correlacao = df.corr()
correlacao

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,1.000000,0.011156,-0.021713,-0.009921,-0.033226,-0.008916,-0.024071,0.008332,0.013324,-0.014160,0.005847,-0.001648,-0.016517,-0.008645,0.001023,0.005764,-0.028365,0.012609,-0.012713,-0.021998,0.013715,-0.014067,0.007869,0.019169,-0.021465,-0.051071,0.002656,-0.040706,0.017861,-0.043789,-0.013579,0.024859,-0.014830,-0.005024,-0.040488,-0.005968,-0.007940,-0.015415,0.061902,-0.011300,-0.009821,-0.056728,0.010496,0.005590,-0.044230,0.008273,0.002289,-0.020155,0.005587,0.006784,0.037719,0.002951,0.004067,0.027239,-0.010104,-0.019772,-0.024325,0.002256,-0.000122,0.011390,0.016570,0.017634,-0.014586,0.000987,0.008921,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.020018,-0.016834,-0.021917
MSSubClass,0.011156,1.000000,0.280306,-0.356718,-0.139781,0.024969,-0.115361,-0.025090,-0.022844,-0.037573,-0.025672,0.288623,-0.047637,0.048746,0.781224,0.355074,0.032628,-0.059316,0.027850,0.040581,-0.088609,-0.011446,0.031824,0.063106,-0.006392,0.023573,0.032483,0.010638,-0.027459,0.068937,0.009467,0.035369,-0.101964,-0.069836,-0.024563,-0.065649,-0.140759,-0.238518,0.031321,0.008709,0.101774,-0.027321,-0.251758,0.307886,0.046474,0.074853,0.003491,-0.002333,0.131608,0.177354,-0.023438,0.281721,0.003478,0.040380,0.001646,-0.045569,-0.003031,0.131387,0.081396,-0.012301,-0.040110,-0.098672,0.030199,0.004600,0.025973,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,0.014963,0.004788,-0.084284
MSZoning,-0.021713,0.280306,1.000000,-0.239076,-0.164093,0.115502,-0.188404,0.008574,-0.011701,-0.082953,-0.019212,0.074866,-0.027378,0.016262,0.220096,0.137691,-0.093649,-0.017370,-0.166358,-0.041200,-0.051533,-0.043703,0.056285,0.006916,-0.121612,-0.070805,0.064779,0.056790,0.081504,-0.026911,0.082340,-0.072682,-0.074661,-0.167706,-0.075849,-0.086492,0.001409,-0.204741,0.030069,0.026429,0.214583,0.108924,-0.252797,0.100493,0.101754,-0.093081,-0.129339,-0.060495,-0.058391,-0.028414,-0.106554,0.086998,-0.000931,-0.105842,-0.009408,-0.211995,-0.127785,0.143970,-0.042682,-0.141796,-0.122763,-0.097245,0.056752,0.047827,0.166244,-0.149897,0.083331,0.124006,-0.046646,-0.063773,-0.030701,-0.017745,0.030967,-0.002833,0.072558,0.099049,-0.183642
LotFrontage,-0.009921,-0.356718,-0.239076,1.000000,0.304522,0.037349,0.191420,0.084287,-0.001027,0.046862,0.065603,-0.179733,0.087120,0.039035,-0.302369,-0.082860,0.234812,-0.053281,0.116685,0.083348,0.138356,0.178043,-0.037647,-0.041864,0.125250,0.178469,0.164055,-0.051685,-0.080647,0.110130,-0.056089,0.133328,-0.009536,0.214367,0.009612,0.042463,0.124098,0.363472,-0.019027,-0.087182,-0.067350,-0.063630,0.413773,0.072388,0.037469,0.368007,0.090343,-0.006979,0.180534,0.047222,0.236840,-0.004905,0.155396,0.320518,-0.028554,0.233221,0.072294,-0.065704,0.062996,0.211125,0.269539,0.323511,-0.038252,-0.033663,-0.067186,0.075542,0.137014,0.010287,0.061945,0.037655,0.180819,-0.000255,0.010451,0.006380,0.015502,0.018490,0.334771
LotArea,-0.033226,-0.139781,-0.164093,0.304522,1.000000,0.197131,0.315484,0.339008,0.010123,0.131099,0.436868,0.068220,0.035510,0.034307,-0.091605,-0.024038,0.105806,-0.005636,0.014228,0.013788,0.106159,0.174461,0.0670

# (Antes de conhecer Select K Best)Função para separar as colunas com maiores correlações

In [121]:
linhas_indice = correlacao.index
def maiores_corr(minimo):
  maiores = []
  for x in range(len(correlacao['SalePrice'])):
    if abs(correlacao['SalePrice'].iloc[x]) > minimo and abs(correlacao['SalePrice'].iloc[x]) < 1:
      maiores.append(linhas_indice[x])
  print('ok')
  return maiores
 

In [151]:
col = maiores_corr(0.6)
col

ok


['OverallQual',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'GarageCars',
 'GarageArea']

#Select K Best

In [477]:
#Funçao seleciona as melhores features de acorco com Select K Beast e treina o modelo
# dataset = é o dataframe o qual será selecionada as features
# quant_col = quantidade K de features selecionadas, no qual serão selecionados grupos de colunas 
# de 1 até quant_col
# alvo = nome da coluna target do dataframe
# tam_test = quantidade de variáveis separadas para teste
# min_escore = é o mínimo de escore de saída
# a função printa na tela as colunas e os escores maiores que o min_score
# a função retorna três variáveis: O maior score, as olunas e o modelo que gerou o maior escore.

def m_features(dataset, quant_col,alvo,tam_test,min_escore):

  from sklearn.feature_selection import SelectKBest
  from sklearn.linear_model import LinearRegression
  from sklearn.model_selection import train_test_split
  #
  x = dataset.drop(columns= alvo ,axis = 1)
  y = dataset[alvo]
  maior_score = 0
  colunas = []
  m_modelo = None

  for i in range(1,quant_col + 1):
    from sklearn.feature_selection import f_classif, mutual_info_classif
    f_classif = SelectKBest(score_func=f_classif, k= i)
    fit = f_classif.fit(x,y)
    indices = fit.get_support(indices=True)
    cols = dataset.columns[indices]

    X = dataset[cols]
  
    X_treino, X_teste, y_treino, y_teste = train_test_split(X,y, test_size = tam_test)
    modelo = LinearRegression()
    modelo.fit(X_treino, y_treino)
    escore = modelo.score(X_teste, y_teste)
    if escore > min_escore:
      print(f'Com as {i} colunas: {cols}, o score foi {escore}\n')
      if escore > maior_score:
        maior_score = escore
        colunas = cols
        m_modelo = modelo
  funcao = 'y = ' + str(modelo.intercept_) +' + '
  coef = m_modelo.coef_
  for c in range(len(coef)):
    funcao += str(coef[c]) + f'x{c+ 1} '
  
  print(34*'-',' FIM ',34*'-','\n')
  print(f'O modelo de saída alcançou score de {maior_score} e possui {len(colunas)} colunas.\n')
  print(33*'-',' Colunas ',33*'-','\n')
  print(colunas)
  print('\n',33*'-',' Função ',33*'-','\n')
  print(funcao)
  print('\n',80*'-')
  

  return maior_score, colunas, m_modelo
    
   

In [478]:
escore, col, modelo = m_features(df,50, 'SalePrice', 0.30, 0.85)


Com as 34 colunas: Index(['MSZoning', 'LotArea', 'Street', 'LotShape', 'OverallQual', 'YearBuilt',
       'YearRemodAdd', 'Exterior1st', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'Foundation', 'BsmtQual', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'MiscVal'],
      dtype='object'), o score foi 0.8750426211779614

Com as 38 colunas: Index(['MSZoning', 'LotArea', 'Street', 'LotShape', 'OverallQual', 'YearBuilt',
       'YearRemodAdd', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure',
       'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath',
       'HalfBath', 'BedroomAbvGr'

#Preparação do dataset de teste para envio da predição

In [129]:
dicionario_nulos = dict_nulos(df_t)
dicionario_nulos


{'Alley': 1352,
 'Fence': 1169,
 'FireplaceQu': 730,
 'MiscFeature': 1408,
 'PoolQC': 1456}

In [130]:
categorico, nao_categorico = sep_cat(dicionario_nulos,df_t)

ok


In [131]:
preencher_nulos_cat(df_t,categorico)

In [132]:
preencher_nulos_nao_cat(df_t,nao_categorico)

In [133]:
dicionario_nulos = dict_nulos(df)
dicionario_nulos

{}

In [134]:
categorico_total = sep_cat_total(df_t)

ok


In [135]:
classif_categoria(df_t,categorico_total)

ok


In [136]:
df_t

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,4,80.0,11622,0,0.0,0,0,0,0,0,0,1,0,0,0,5,6,1961,1961,0,0,0,0,0,0.0,0,0,1,0,0,0,3,468.0,2,144.0,270.0,882.0,0,...,0,0,896,0,0,896,0.0,0.0,1,0,2,1,0,5,0,0,1.0,0,1961.0,0,1.0,730.0,0,0,0,140,0,0,0,120,0,0.0,0.0,0.0,0,6,2010,0,0,109086.982239
1,1462,20,0,81.0,14267,0,0.0,1,0,0,1,0,0,0,0,0,0,6,6,1958,1958,1,0,3,3,1,108.0,0,0,1,0,0,0,2,923.0,0,0.0,406.0,1329.0,0,...,0,0,1329,0,0,1329,0.0,0.0,1,1,3,1,1,6,0,0,1.0,0,1958.0,0,1.0,312.0,0,0,0,393,36,0,0,0,0,0.0,0.0,1.0,12500,6,2010,0,0,149333.230861
2,1463,60,0,74.0,13830,0,0.0,1,0,0,0,0,6,0,0,0,1,5,5,1997,1998,0,0,0,0,0,0.0,0,0,0,1,0,0,0,791.0,0,0.0,137.0,928.0,0,...,0,0,928,701,0,1629,0.0,0.0,2,1,3,1,0,6,0,1,1.0,0,1997.0,2,2.0,482.0,0,0,0,212,34,0,0,0,0,0.0,0.0,0.0,0,3,2010,0,0,170397.189540
3,1464,60,0,78.0,9978,0,0.0,1,0,0,0,0,6,0,0,0,1,6,6,1998,1998,0,0,0,0,1,20.0,0,0,0,0,0,0,0,602.0,0,0.0,324.0,926.0,0,...,0,0,926,678,0,1604,0.0,0.0,2,1,3,1,1,7,0,1,0.0,0,1998.0,2,2.0,470.0,0,0,0,360,36,0,0,0,0,0.0,0.0,0.0,0,6,2010,0,0,189185.335361
4,1465,120,0,43.0,5005,0,0.0,1,1,0,0,0,16,0,0,1,0,8,5,1992,1992,0,0,2,2,0,0.0,1,0,0,1,0,0,2,263.0,0,0.0,1017.0,1280.0,0,...,0,0,1280,0,0,1280,0.0,0.0,2,0,2,1,1,5,0,0,1.0,0,1992.0,1,2.0,506.0,0,0,0,0,82,0,0,144,0,0.0,0.0,0.0,0,1,2010,0,0,220726.444895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,1,21.0,1936,0,0.0,0,0,0,0,0,18,0,0,3,1,4,7,1970,1970,0,0,5,5,0,0.0,0,0,1,0,0,0,1,0.0,0,0.0,546.0,546.0,0,...,0,0,546,546,0,1092,0.0,0.0,1,1,3,1,0,5,0,0,1.0,0,1979.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,6,2006,0,0,70355.649127
1455,2916,160,1,21.0,1894,0,0.0,0,0,0,0,0,18,0,0,1,1,4,5,1970,1970,0,0,5,5,0,0.0,0,0,1,0,0,0,3,252.0,0,0.0,294.0,546.0,0,...,0,0,546,546,0,1092,0.0,0.0,1,1,3,1,0,6,0,0,1.0,5,1970.0,0,1.0,286.0,0,0,0,0,24,0,0,0,0,0.0,0.0,0.0,0,4,2006,0,2,87599.978532
1456,2917,20,0,160.0,20000,0,0.0,0,0,0,0,0,9,0,0,0,0,5,7,1960,1996,0,0,0,0,0,0.0,0,0,1,0,0,0,2,1224.0,0,0.0,0.0,1224.0,0,...,0,0,1224,0,0,1224,1.0,0.0,1,0,4,1,0,7,0,1,1.0,1,1960.0,0,2.0,576.0,0,0,0,474,0,0,0,0,0,0.0,0.0,0.0,0,9,2006,0,2,148712.780790
1457,2918,85,0,62.0,10441,0,0.0,0,0,0,0,0,9,0,0,0,4,5,5,1992,1992,0,0,2,6,0,0.0,0,0,0,1,0,1,0,337.0,0,0.0,575.0,912.0,0,...,0,0,970,0,0,970,0.0,1.0,1,0,3,1,0,6,0,0,1.0,0,1979.0,0,0.0,0.0,0,0,0,80,32,0,0,0,0,0.0,0.0,0.0,700,7,2006,0,0,102884.791574


In [460]:
prev = modelo.predict(df_t[col])

In [461]:
df_t['SalePrice'] = prev

In [462]:
submission = df_t.filter(['Id','SalePrice'])

In [463]:
submission.to_csv('submission.csv', index=False)